# Update metadata file

This script allows to update the metadata file
- Get new list of series

## Load necessary libraries

In [29]:
import csv
import json
import urllib3  # allows to access a URL with python
import math
import os
import io
import collections
import copy
import numpy as np
import pandas as pd
import re
from collections import OrderedDict
import xlsxwriter

# https://volderette.de/jupyter-notebook-tip-multiple-outputs/
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Working directory

In [7]:
dir_path = os.path.dirname(os.path.realpath('__file__'))
print(dir_path)

wd_dir = r'../'
print('Main dir: ' + wd_dir)

C:\Users\L.GonzalezMorales\Documents\GitHub\FIS4SDGs\unsd\notebooks
Main dir: ../


## Utilities

#### Convert string to camelCase

In [8]:
def camelCase(st):
    """
    https://stackoverflow.com/questions/8347048/camelcase-every-string-any-standard-library
    
    """
    output = ''.join(x for x in st.title() if x.isalnum())
    return output[0].lower() + output[1:]

## Read old `metadata.json`

In [9]:
json_file = open(wd_dir + 'metadata.json')
json_str = json_file.read()
metadata_old = json.loads(json_str)
metadata_old[0]

{'goalCode': 1,
 'goalDescription': 'End poverty in all its forms everywhere',
 'targetCode': '1.1',
 'targetDescription': 'By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day',
 'indicatorCode': '1.1.1',
 'indicatorDescription': 'Proportion of population below the international poverty line, by sex, age, employment status and geographical location (urban/rural)',
 'seriesCode': 'SI_POV_DAY1',
 'seriesDescription': 'Proportion of population below international poverty line (%)',
 'seriesRelease': '2018.Q2.G.01',
 'TAGS': ['poverty line', 'poverty', 'standard of living', 'basic needs'],
 'hex': 'e5243b',
 'rgb': [229, 36, 59],
 'iconUrl': 'https://raw.githubusercontent.com/UNStats/FIS4SDGs/master/globalResources/sdgIcons/SDG01.png',
 'ColorScheme': ['FFDA99',
  'FAC590',
  'F5967F',
  'F1786E',
  'ED5C5E',
  'E5233D',
  'BF162F',
  '93071F'],
 'ColorSchemeCredits': "Prepared by Sarah Bell, Esri's Cartography Lab, 27 

### Extract icons and color scheme for each of 17 goals

In [10]:
sdg_colors = []
goalCode = None

for m in metadata_old:
    
    if str(m['goalCode']) == goalCode:
        continue

    goalCode = str(m['goalCode'])
    
    dict = {}
    
    dict['goalCode'] = goalCode
    dict['hex'] = m['hex']
    dict['rgb'] = m['rgb']
    dict['iconUrl'] = m['iconUrl'] 
    dict['ColorScheme'] = m['ColorScheme']
    dict['ColorSchemeCredits'] = m['ColorSchemeCredits']
    
    sdg_colors.append(dict)
    
sdg_colors[0]
    
    
    
    
    
    

{'goalCode': '1',
 'hex': 'e5243b',
 'rgb': [229, 36, 59],
 'iconUrl': 'https://raw.githubusercontent.com/UNStats/FIS4SDGs/master/globalResources/sdgIcons/SDG01.png',
 'ColorScheme': ['FFDA99',
  'FAC590',
  'F5967F',
  'F1786E',
  'ED5C5E',
  'E5233D',
  'BF162F',
  '93071F'],
 'ColorSchemeCredits': "Prepared by Sarah Bell, Esri's Cartography Lab, 27 July 2017"}

## Read `file_metadata.json`

This json file was produced with script `FIS4SDG - 02 - fileMetadataipynb` and contains the description of latest dataset files extracted from the SDG API

In [12]:
json_file = open(wd_dir + 'data/file_metadata.json')
json_str = json_file.read()
dataset_metadata = json.loads(json_str)
dataset_metadata[0].keys()

dict_keys(['fileName', 'goalCode', 'goalDesc', 'targetCode', 'targetDesc', 'indicatorCode', 'indicatorDesc', 'indicatorTier', 'seriesCode', 'seriesDesc', 'seriesRelease', 'slices', 'time_coverage', 'geo_coverage_groups', 'geo_coverage_regions', 'geo_coverage_countries'])

### Create a metadata template file for the current data release

In [14]:
series_metadata = []
for s in dataset_metadata:
    
    dict = {}
    
    dict['goalCode'] = s['goalCode']
    dict['goalDesc'] = s['goalDesc']
    dict['goalDesc'] = s['goalDesc']
    dict['targetCode'] = s['targetCode']
    dict['targetDesc'] = s['targetDesc']
    dict['indicatorCode'] = s['indicatorCode']
    dict['indicatorDesc'] = s['indicatorDesc']
    dict['indicatorTier'] = s['indicatorTier']
    dict['seriesCode'] = s['seriesCode']
    dict['seriesDesc'] = s['seriesDesc']
    dict['seriesRelease'] = s['seriesRelease']
    
    dict['TAGS'] = []
    for m in metadata_old:
        if m['seriesCode'] ==  s['seriesCode'] and  m['indicatorCode'] ==  s['indicatorCode']:
            dict['TAGS'] = m['TAGS']
            break

    for g in sdg_colors:
        if g['goalCode'] == s['goalCode']:
                dict['hex'] = g['hex']
                dict['rgb'] = g['rgb']
                dict['iconUrl'] = g['iconUrl']
                dict['ColorScheme'] = g['ColorScheme']
                dict['ColorSchemeCredits'] = g['ColorSchemeCredits']
                break
                
    dict['slices'] = s['slices']
    dict['time_coverage'] = s['time_coverage']
    dict['geo_coverage_groups'] = s['geo_coverage_groups']
    dict['geo_coverage_regions'] = s['geo_coverage_regions']
    dict['geo_coverage_countries'] = s['geo_coverage_countries']
    
    
    
    series_metadata.append(OrderedDict(dict))

#series_metadata[0]

sm = pd.DataFrame.from_records(series_metadata) 

sm.head(3)

sm.to_excel(wd_dir + 'metadata_template.xlsx', engine ='xlsxwriter', index=False)

    

,goalCode,goalDesc,targetCode,targetDesc,indicatorCode,indicatorDesc,indicatorTier,seriesCode,seriesDesc,seriesRelease,...,hex,rgb,iconUrl,ColorScheme,ColorSchemeCredits,slices,time_coverage,geo_coverage_groups,geo_coverage_regions,geo_coverage_countries
0,1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",1.1.1,Proportion of population below the internation...,1,SI_POV_DAY1,Proportion of population below international p...,2019.Q1.G.02,...,e5243b,"[229, 36, 59]",https://raw.githubusercontent.com/UNStats/FIS4...,"[FFDA99, FAC590, F5967F, F1786E, ED5C5E, E5233...","Prepared by Sarah Bell, Esri's Cartography Lab...","[{'reportingTypeCode': 'G', 'seriesCode': 'SI_...","[1990, 1991, 1992, 1993, 1994, 1995, 1996, 199...","[{'geoAreaCode': '910', 'geoAreaName': 'High i...","[{'geoAreaCode': '1', 'geoAreaName': 'World'}]","[{'geoAreaCode': '8', 'geoAreaName': 'Albania'..."
1,1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",1.1.1,Proportion of population below the internation...,1,SI_POV_EMP1,Employed population below international povert...,2019.Q1.G.02,...,e5243b,"[229, 36, 59]",https://raw.githubusercontent.com/UNStats/FIS4...,"[FFDA99, FAC590, F5967F, F1786E, ED5C5E, E5233...","Prepared by Sarah Bell, Esri's Cartography Lab...","[{'ageCode': '15+', 'reportingTypeCode': 'G', ...","[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200...","[{'geoAreaCode': '199', 'geoAreaName': 'Least ...","[{'geoAreaCode': '1', 'geoAreaName': 'World'},...","[{'geoAreaCode': '8', 'geoAreaName': 'Albania'..."
2,1,End poverty in all its forms everywhere,1.2,"By 2030, reduce at least by half the proportio...",1.2.1,Proportion of population living below the nati...,1,SI_POV_NAHC,Proportion of population living below the nati...,2019.Q1.G.02,...,e5243b,"[229, 36, 59]",https://raw.githubusercontent.com/UNStats/FIS4...,"[FFDA99, FAC590, F5967F, F1786E, ED5C5E, E5233...","Prepared by Sarah Bell, Esri's Cartography Lab...","[{'locationCode': 'ALLAREA', 'reportingTypeCod...","[1985, 1987, 1989, 1990, 1991, 1992, 1993, 199...",[],[],"[{'geoAreaCode': '4', 'geoAreaName': 'Afghanis..."


In [37]:
series_metadata[0]['seriesCode']
series_metadata[0]['TAGS']

f = 'metadata_template_edited.xlsx'

xlsx = pd.ExcelFile(wd_dir + f)
x = pd.read_excel(xlsx, 'Sheet1')
x.shape
x.head(3)
x.columns

for index, row in x.iterrows():
    seriesCode = row['seriesCode']
    tags = row['TAGS']
    tags = re.sub('[\[\]\']', '', tags)
    tags = tags.split(",")
    tags = list(map(str.lstrip, tags))
    print(tags)
    



'SI_POV_DAY1'

['poverty line', 'poverty', 'standard of living', 'basic needs']

(429, 21)

,goalCode,goalDesc,targetCode,targetDesc,indicatorCode,indicatorDesc,indicatorTier,seriesCode,seriesDesc,seriesRelease,...,hex,rgb,iconUrl,ColorScheme,ColorSchemeCredits,slices,time_coverage,geo_coverage_groups,geo_coverage_regions,geo_coverage_countries
0,1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",1.1.1,Proportion of population below the internation...,1,SI_POV_DAY1,Proportion of population below international p...,2019.Q1.G.02,...,e5243b,"[229, 36, 59]",https://raw.githubusercontent.com/UNStats/FIS4...,"['FFDA99', 'FAC590', 'F5967F', 'F1786E', 'ED5C...","Prepared by Sarah Bell, Esri's Cartography Lab...","[{'reportingTypeCode': 'G', 'seriesCode': 'SI_...","['1990', '1991', '1992', '1993', '1994', '1995...","[{'geoAreaCode': '910', 'geoAreaName': 'High i...","[{'geoAreaCode': '1', 'geoAreaName': 'World'}]","[{'geoAreaCode': '8', 'geoAreaName': 'Albania'..."
1,1,End poverty in all its forms everywhere,1.1,"By 2030, eradicate extreme poverty for all peo...",1.1.1,Proportion of population below the internation...,1,SI_POV_EMP1,Employed population below international povert...,2019.Q1.G.02,...,e5243b,"[229, 36, 59]",https://raw.githubusercontent.com/UNStats/FIS4...,"['FFDA99', 'FAC590', 'F5967F', 'F1786E', 'ED5C...","Prepared by Sarah Bell, Esri's Cartography Lab...","[{'ageCode': '15+', 'reportingTypeCode': 'G', ...","['2000', '2001', '2002', '2003', '2004', '2005...","[{'geoAreaCode': '199', 'geoAreaName': 'Least ...","[{'geoAreaCode': '1', 'geoAreaName': 'World'},...","[{'geoAreaCode': '8', 'geoAreaName': 'Albania'..."
2,1,End poverty in all its forms everywhere,1.2,"By 2030, reduce at least by half the proportio...",1.2.1,Proportion of population living below the nati...,1,SI_POV_NAHC,Proportion of population living below the nati...,2019.Q1.G.02,...,e5243b,"[229, 36, 59]",https://raw.githubusercontent.com/UNStats/FIS4...,"['FFDA99', 'FAC590', 'F5967F', 'F1786E', 'ED5C...","Prepared by Sarah Bell, Esri's Cartography Lab...","[{'locationCode': 'ALLAREA', 'reportingTypeCod...","['1985', '1987', '1989', '1990', '1991', '1992...",[],[],"[{'geoAreaCode': '4', 'geoAreaName': 'Afghanis..."


Index(['goalCode', 'goalDesc', 'targetCode', 'targetDesc', 'indicatorCode',
       'indicatorDesc', 'indicatorTier', 'seriesCode', 'seriesDesc',
       'seriesRelease', 'TAGS', 'hex', 'rgb', 'iconUrl', 'ColorScheme',
       'ColorSchemeCredits', 'slices', 'time_coverage', 'geo_coverage_groups',
       'geo_coverage_regions', 'geo_coverage_countries'],
      dtype='object')

['poverty line', 'poverty', 'standard of living', 'basic needs']
['poverty line', 'poverty', 'standard of living', 'basic needs', 'decent work']
['poverty line', 'poverty', 'standard of living', 'basic needs']
['poverty', 'standard of living', 'basic needs', 'social welfare']
['poverty', 'standard of living', 'basic needs', 'social welfare', 'child welfare', 'children']
['poverty', 'standard of living', 'basic needs', 'social welfare', 'disability benefits', 'persons with disabilities']
['poverty', 'standard of living', 'basic needs', 'social welfare']
['poverty', 'standard of living', 'basic needs', 'social welfare']
['poverty', 'standard of living', 'basic needs', 'social welfare', 'child welfare', 'family welfare', 'maternal welfare', 'women', 'mothers', 'children']
['poverty', 'standard of living', 'basic needs', 'social welfare', 'old age benefits', 'ageing persons']
['poverty', 'standard of living', 'basic needs', 'social welfare', 'insurance']
['poverty', 'standard of living', '

#### Create list of target icons

In [10]:
files = os.listdir(wd_dir + r'globalResources/sdgTargets/')
target_icons = []
for f in files:
    dict = {}
    x = f.split('_')
    target = x[3][:len(x[3])-4]
    goal = x[1]
    dict['goalCode'] = goal
    dict['targetCode'] = target
    dict['iconFile'] = f
    
    target_icons.append(dict)
    
target_icons[0:5]

[{'goalCode': '10',
  'targetCode': '10.1',
  'iconFile': 'GOAL_10_TARGET_10.1.png'},
 {'goalCode': '10',
  'targetCode': '10.2',
  'iconFile': 'GOAL_10_TARGET_10.2.png'},
 {'goalCode': '10',
  'targetCode': '10.3',
  'iconFile': 'GOAL_10_TARGET_10.3.png'},
 {'goalCode': '10',
  'targetCode': '10.4',
  'iconFile': 'GOAL_10_TARGET_10.4.png'},
 {'goalCode': '10',
  'targetCode': '10.5',
  'iconFile': 'GOAL_10_TARGET_10.5.png'}]

### Read `json` with file information

In [11]:
json_file = open(wd_dir + 'unsd/data/file_metadata.json')
json_str = json_file.read()
metadata_2 = json.loads(json_str)
metadata_2[8]

{'fileName': 'Data_1.3.1_SI_COV_MATNL.xlsx',
 'goalCode': '1',
 'goalDesc': 'End poverty in all its forms everywhere',
 'targetCode': '1.3',
 'targetDesc': 'Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable',
 'indicatorCode': '1.3.1',
 'indicatorDesc': 'Proportion of population covered by social protection floors/systems, by sex, distinguishing children, unemployed persons, older persons, persons with disabilities, pregnant women, newborns, work-injury victims and the poor and the vulnerable',
 'indicatorTier': '1',
 'seriesCode': 'SI_COV_MATNL',
 'seriesDesc': '[ILO] Proportion of mothers with newborns receiving maternity cash benefit (%)',
 'seriesRelease': '2019.Q1.G.02',
 'slices': [{'reportingTypeCode': 'G',
   'seriesCode': 'SI_COV_MATNL',
   'sexCode': 'BOTHSEX'},
  {'reportingTypeCode': 'G',
   'seriesCode': 'SI_COV_MATNL',
   'sexCode': 'FEMALE'}],
 'time_c